# Portfolio Construction and Multi-Strategy Backtesting

This notebook demonstrates how to:
1. Combine multiple strategies into a portfolio
2. Test different allocation methods (equal weight, inverse volatility)
3. Analyze portfolio performance and correlations
4. Compare with individual strategy performance

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path
sys.path.append('..')

# Import our modules
from src.data.preprocessor import DataPreprocessor
from src.data.features import FeatureEngine
from src.strategies.examples.moving_average import MovingAverageCrossover
from src.strategies.examples.orb import OpeningRangeBreakout
from src.backtesting.engines.vectorbt_engine import VectorBTEngine
from src.backtesting.portfolio import PortfolioBacktester
from src.analysis import PerformanceAnalyzer, StrategyVisualizer

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Data for Multiple Symbols

In [ ]:
# Define symbols to use in portfolio
symbols = ['AAPL', 'MSFT', 'SPY', 'QQQ']
months = ['2024_01', '2024_02']  # Use 2 months for faster demo

# Initialize preprocessor and feature engine
preprocessor = DataPreprocessor(
    raw_data_dir=Path('../data/raw/minute_aggs/by_symbol'),
    processed_data_dir=Path('../data/processed'),
    cache_dir=Path('../data/cache')
)
feature_engine = FeatureEngine()

# Load and process data for each symbol
symbol_data = {}
for symbol in symbols:
    print(f"Loading {symbol}...")
    try:
        # Process data
        processed = preprocessor.process(symbol, months)
        # Add features
        data_with_features = feature_engine.add_all_features(processed)
        symbol_data[symbol] = data_with_features
        print(f"  Loaded {len(data_with_features)} bars")
    except Exception as e:
        print(f"  Error loading {symbol}: {e}")

print(f"\nSuccessfully loaded data for {len(symbol_data)} symbols")

## 2. Define Strategies for Each Symbol

In [ ]:
# Define strategy configurations
strategy_configs = [
    # Moving Average strategies
    ('MA_Fast', MovingAverageCrossover, {
        'fast_period': 10,
        'slow_period': 30,
        'ma_type': 'ema'
    }),
    ('MA_Slow', MovingAverageCrossover, {
        'fast_period': 20,
        'slow_period': 50,
        'ma_type': 'sma'
    }),
    # ORB strategies
    ('ORB_5min', OpeningRangeBreakout, {
        'range_minutes': 5,
        'profit_target_r': 3.0,
        'stop_type': 'range'
    }),
    ('ORB_15min', OpeningRangeBreakout, {
        'range_minutes': 15,
        'profit_target_r': 5.0,
        'stop_type': 'atr'
    })
]

# Create strategies dictionary
strategies = {}
for symbol in symbol_data.keys():
    for strat_name, strat_class, params in strategy_configs:
        key = f"{symbol}_{strat_name}"
        strategies[key] = strat_class(params)
        
print(f"Created {len(strategies)} total strategy-symbol combinations")
print("\nStrategy keys:")
for i, key in enumerate(strategies.keys()):
    print(f"  {i+1}. {key}")
    if i >= 9:  # Show first 10
        print(f"  ... and {len(strategies) - 10} more")
        break

## 3. Backtest Individual Strategies

In [ ]:
# Initialize backtesting engine
engine = VectorBTEngine(freq='1min')

# Run backtests for all strategies
backtest_results = {}
initial_capital = 100000

print("Running individual backtests...")
for key, strategy in strategies.items():
    symbol = key.split('_')[0]
    if symbol in symbol_data:
        print(f"  Backtesting {key}...", end=' ')
        try:
            result = engine.run_backtest(
                strategy=strategy,
                data=symbol_data[symbol],
                initial_capital=initial_capital,
                commission=0.0005,
                slippage=0.0001
            )
            backtest_results[key] = result
            print(f"Sharpe: {result.metrics['sharpe_ratio']:.2f}, Return: {result.metrics['total_return']:.1%}")
        except Exception as e:
            print(f"Error: {e}")

print(f"\nCompleted {len(backtest_results)} backtests")

## 4. Analyze Individual Strategy Performance

In [ ]:
# Create performance summary
performance_data = []
for key, result in backtest_results.items():
    performance_data.append({
        'Strategy': key,
        'Symbol': key.split('_')[0],
        'Type': key.split('_')[1],
        'Sharpe': result.metrics['sharpe_ratio'],
        'Return': result.metrics['total_return'],
        'MaxDD': result.metrics['max_drawdown'],
        'WinRate': result.metrics.get('win_rate', 0)
    })

performance_df = pd.DataFrame(performance_data)
performance_df = performance_df.sort_values('Sharpe', ascending=False)

print("Top 10 Individual Strategies:")
print(performance_df.head(10).to_string(index=False))

# Plot performance by strategy type
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sharpe ratios by strategy type
performance_df.boxplot(column='Sharpe', by='Type', ax=axes[0])
axes[0].set_title('Sharpe Ratio by Strategy Type')
axes[0].set_xlabel('Strategy Type')
axes[0].set_ylabel('Sharpe Ratio')

# Returns by symbol
performance_df.boxplot(column='Return', by='Symbol', ax=axes[1])
axes[1].set_title('Returns by Symbol')
axes[1].set_xlabel('Symbol')
axes[1].set_ylabel('Total Return')

plt.tight_layout()
plt.show()

## 5. Create Multi-Strategy Portfolios

In [ ]:
# Select top strategies for portfolio
top_strategies = performance_df.head(8)['Strategy'].tolist()
print(f"Selected top {len(top_strategies)} strategies for portfolio:")
for i, strat in enumerate(top_strategies):
    print(f"  {i+1}. {strat}")

# Create portfolio backtester
portfolio_backtester = PortfolioBacktester()

# Prepare strategies and data for portfolio
portfolio_strategies = {key: strategies[key] for key in top_strategies}
portfolio_data = {}
for key in top_strategies:
    symbol = key.split('_')[0]
    portfolio_data[key] = symbol_data[symbol]

print(f"\nPortfolio contains {len(portfolio_strategies)} strategies")

## 6. Test Different Allocation Methods

In [ ]:
# Test different allocation methods
allocation_methods = ['equal_weight', 'inverse_volatility']
portfolio_results = {}

for method in allocation_methods:
    print(f"\nTesting {method} allocation...")
    
    try:
        result = portfolio_backtester.run_portfolio_backtest(
            strategies=portfolio_strategies,
            data=portfolio_data,
            initial_capital=initial_capital,
            allocation_method=method,
            rebalance_frequency='monthly'
        )
        
        portfolio_results[method] = result
        
        print(f"  Portfolio Sharpe: {result.portfolio_metrics['sharpe_ratio']:.2f}")
        print(f"  Portfolio Return: {result.portfolio_metrics['total_return']:.1%}")
        print(f"  Portfolio MaxDD: {result.portfolio_metrics['max_drawdown']:.1%}")
        
    except Exception as e:
        print(f"  Error: {e}")

# Add custom weighted portfolio
custom_weights = {
    key: 1.0 / len(top_strategies) for key in top_strategies
}
# Give more weight to top performers
if len(top_strategies) >= 2:
    custom_weights[top_strategies[0]] = 0.3  # 30% to best
    custom_weights[top_strategies[1]] = 0.2  # 20% to second best
    # Distribute remaining 50% equally
    remaining = 0.5 / (len(top_strategies) - 2) if len(top_strategies) > 2 else 0
    for key in top_strategies[2:]:
        custom_weights[key] = remaining

print("\nTesting custom weighted allocation...")
try:
    result = portfolio_backtester.run_portfolio_backtest(
        strategies=portfolio_strategies,
        data=portfolio_data,
        initial_capital=initial_capital,
        allocation_method='custom',
        custom_weights=custom_weights,
        rebalance_frequency='monthly'
    )
    
    portfolio_results['custom_weighted'] = result
    
    print(f"  Portfolio Sharpe: {result.portfolio_metrics['sharpe_ratio']:.2f}")
    print(f"  Portfolio Return: {result.portfolio_metrics['total_return']:.1%}")
    print(f"  Portfolio MaxDD: {result.portfolio_metrics['max_drawdown']:.1%}")
    
except Exception as e:
    print(f"  Error: {e}")

## 7. Compare Portfolio vs Individual Performance

In [ ]:
# Create comparison dataframe
comparison_data = []

# Add individual strategies
for key in top_strategies:
    if key in backtest_results:
        result = backtest_results[key]
        comparison_data.append({
            'Strategy': key,
            'Type': 'Individual',
            'Sharpe': result.metrics['sharpe_ratio'],
            'Return': result.metrics['total_return'],
            'MaxDD': result.metrics['max_drawdown'],
            'Volatility': result.metrics.get('volatility', 0)
        })

# Add portfolio results
for method, result in portfolio_results.items():
    comparison_data.append({
        'Strategy': f'Portfolio_{method}',
        'Type': 'Portfolio',
        'Sharpe': result.portfolio_metrics['sharpe_ratio'],
        'Return': result.portfolio_metrics['total_return'],
        'MaxDD': result.portfolio_metrics['max_drawdown'],
        'Volatility': result.portfolio_metrics.get('volatility', 0)
    })

comparison_df = pd.DataFrame(comparison_data)

# Display comparison
print("Portfolio vs Individual Strategy Performance:")
print(comparison_df.sort_values('Sharpe', ascending=False).to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sharpe Ratio comparison
comparison_df.plot(x='Strategy', y='Sharpe', kind='bar', ax=axes[0,0], color=['blue' if t == 'Individual' else 'red' for t in comparison_df['Type']])
axes[0,0].set_title('Sharpe Ratio Comparison')
axes[0,0].set_xticklabels(comparison_df['Strategy'], rotation=45, ha='right')
axes[0,0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Return comparison
comparison_df.plot(x='Strategy', y='Return', kind='bar', ax=axes[0,1], color=['blue' if t == 'Individual' else 'red' for t in comparison_df['Type']])
axes[0,1].set_title('Total Return Comparison')
axes[0,1].set_xticklabels(comparison_df['Strategy'], rotation=45, ha='right')
axes[0,1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

# Risk-Return scatter
for idx, row in comparison_df.iterrows():
    color = 'blue' if row['Type'] == 'Individual' else 'red'
    marker = 'o' if row['Type'] == 'Individual' else 's'
    axes[1,0].scatter(row['Volatility'], row['Return'], c=color, marker=marker, s=100)
    axes[1,0].annotate(row['Strategy'].split('_')[-1], (row['Volatility'], row['Return']), fontsize=8)

axes[1,0].set_xlabel('Volatility')
axes[1,0].set_ylabel('Return')
axes[1,0].set_title('Risk-Return Profile')
axes[1,0].grid(True, alpha=0.3)

# Drawdown comparison
comparison_df['MaxDD_abs'] = comparison_df['MaxDD'].abs()
comparison_df.plot(x='Strategy', y='MaxDD_abs', kind='bar', ax=axes[1,1], color=['blue' if t == 'Individual' else 'red' for t in comparison_df['Type']])
axes[1,1].set_title('Maximum Drawdown Comparison')
axes[1,1].set_xticklabels(comparison_df['Strategy'], rotation=45, ha='right')
axes[1,1].set_ylabel('Max Drawdown (absolute)')

plt.tight_layout()
plt.show()

## 8. Analyze Portfolio Composition and Correlations

In [ ]:
# Get the best portfolio result
best_portfolio_method = max(portfolio_results.keys(), key=lambda x: portfolio_results[x].portfolio_metrics['sharpe_ratio'])
best_portfolio = portfolio_results[best_portfolio_method]

print(f"Best portfolio method: {best_portfolio_method}")
print(f"\nAllocation weights:")
for strategy, weight in best_portfolio.allocations.items():
    print(f"  {strategy}: {weight:.1%}")

# Plot correlation matrix
if hasattr(best_portfolio, 'correlation_matrix') and best_portfolio.correlation_matrix is not None:
    plt.figure(figsize=(10, 8))
    
    # Create correlation heatmap
    mask = np.triu(np.ones_like(best_portfolio.correlation_matrix, dtype=bool))
    sns.heatmap(best_portfolio.correlation_matrix, 
                mask=mask,
                annot=True, 
                fmt='.2f',
                cmap='coolwarm',
                center=0,
                square=True,
                linewidths=1,
                cbar_kws={"shrink": 0.8})
    
    plt.title('Strategy Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # Calculate average correlations
    corr_values = best_portfolio.correlation_matrix.values
    upper_triangle = corr_values[np.triu_indices_from(corr_values, k=1)]
    avg_correlation = np.mean(upper_triangle)
    
    print(f"\nAverage correlation between strategies: {avg_correlation:.3f}")
    print(f"Min correlation: {np.min(upper_triangle):.3f}")
    print(f"Max correlation: {np.max(upper_triangle):.3f}")

## 9. Portfolio Equity Curve Analysis

In [ ]:
# Plot equity curves for all portfolio methods
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Plot portfolio equity curves
for method, result in portfolio_results.items():
    if hasattr(result, 'portfolio_equity'):
        result.portfolio_equity.plot(ax=ax1, label=f'Portfolio ({method})', linewidth=2)

# Add best individual strategies for comparison
for i, key in enumerate(top_strategies[:3]):
    if key in backtest_results:
        backtest_results[key].equity_curve.plot(ax=ax1, label=key, alpha=0.5, linestyle='--')

ax1.set_title('Portfolio vs Individual Strategy Equity Curves')
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Plot drawdowns
for method, result in portfolio_results.items():
    if hasattr(result, 'portfolio_equity'):
        equity = result.portfolio_equity
        rolling_max = equity.expanding().max()
        drawdown = ((equity - rolling_max) / rolling_max * 100)
        drawdown.plot(ax=ax2, label=f'Portfolio ({method})', linewidth=2)

ax2.set_title('Portfolio Drawdowns')
ax2.set_xlabel('Date')
ax2.set_ylabel('Drawdown (%)')
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)
ax2.fill_between(drawdown.index, 0, drawdown.values, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Key Takeaways and Next Steps

### Results Summary
1. **Portfolio Benefits**: Multi-strategy portfolios typically show:
   - Lower volatility than individual strategies
   - More consistent returns
   - Reduced maximum drawdowns
   
2. **Allocation Methods**:
   - Equal weight: Simple but effective
   - Inverse volatility: Better risk-adjusted returns
   - Custom weights: Can optimize for specific goals

3. **Diversification**:
   - Combining uncorrelated strategies improves risk-return profile
   - Mix of trend-following (MA) and mean-reversion (ORB) strategies
   
### Next Steps
1. **Expand Strategy Universe**:
   - Add momentum strategies
   - Include pairs trading
   - Test sector rotation
   
2. **Advanced Portfolio Construction**:
   - Implement mean-variance optimization
   - Test risk parity allocation
   - Add dynamic rebalancing based on market conditions
   
3. **Risk Management**:
   - Portfolio-level stop losses
   - Volatility targeting
   - Correlation monitoring and adjustment